In [2]:
import pandas as pd
import numpy as np
import util as u

data='/labs/mpsnyder/moqri/data/index/'
def get_tss():
    ezh_h9='https://ftp.ncbi.nlm.nih.gov/geo/series/GSE76nnn/GSE76626/suppl/GSE76626_ChIP-Seq_RPM.txt.gz' ## hg38
    tss=pd.read_table(ezh_h9,index_col=0,usecols=['Probe','Chromosome','Start','End','WT_EZH2','Strand'])
    tss.columns=['chrom', 'chromStart', 'chromEnd','s','v']
    tss=tss[(tss.chrom!='X') & (tss.chrom!='Y')]
    tss=tss[tss.index.str.contains('001')]
    tss=tss[~tss.index.str.contains('-AS')]
    tss.index=tss.index.str.strip('-001_upstream')
    print(tss.shape)
    tss['d']=tss.chromEnd-tss.chromStart
    tss['tss']=tss.chromStart+2500
    tss['g']=tss.chrom.astype(int)*10**9+tss.tss
    print(tss.d.sum()/1000000,'M')
    tss.to_csv('files/tss38.csv')
def h9_cpg():
    v='38'
    bw_url='http://smithdata.usc.edu/methbase/data/Xie-Human-2013/Human_H9/tracks_hg'+v+'/Human_H9.meth.bw'
    df=u.bw2df(bw_url,tss)
    df['g']=df.index.str.split('_').str[0].astype(int)*10**9+df.index.str.split('_').str[1].astype(int)
    tssl=(tss.chrom.astype(int)*10**9+tss.tss).values
    df['tss']=df.g.apply(lambda y:u.take_closest(tssl, y))
    df['d']=df.g-df.tss
    df['dq']=pd.qcut(df.d,11)
    print(df.shape)
    df.to_csv(data+'wg/cpgs.csv')    
def get_flank():
    df['ch']='chr'+df.index.str.split('_').str[0]
    df['b']=df.index.str.split('_').str[1].astype(int)
    df['b1']=df.b-1
    df['b2']=df.b+3
    df[['ch','b1','b2']].to_csv(data+'wg/flank_pos.bed',sep='\t',header=None,index=None)
    #!bedtools getfasta -fi ref/hg38.fa.masked -bed wg/flank_pos.bed -fo wg/flank.fasta
    flank=pd.read_table(data+'wg/flank.fasta',header=None)
    flank=flank[~flank[0].str.contains('chr')]
    df['flank']=flank[0].values
    flank.shape    

In [3]:
man450='https://zhouserver.research.chop.edu/InfiniumAnnotation/20180909/HM450/HM450.hg38.manifest.tsv.gz'
!wget -nc -q {man450}  -O "{data}ref/man450.tsv.gz"
!gzip -d "{data}ref/man450.tsv.gz"

In [2]:
tss=pd.read_csv('files/tss38.csv',index_col=0)
print(tss.shape)
ezh=tss[tss.v>1]
print(ezh.shape)
df=pd.read_csv(data+'wg/cpgs.csv',index_col=0)
print(df.shape
ezh['gene']=ezh.index
ezh.index=ezh.g
ezh=ezh.gene.to_dict()
ezhs=ezh.keys()
df['ezh']=df.tss.apply(lambda x:ezh[x] if x in ezhs else np.nan)
df.to_csv(data+'wg/ann.csv')    

(31335, 8)
(929, 8)
